## Baseline using Yolov8n


In [15]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("../crowd.mp4")
ret, frame = cap.read()
if ret:
    results = model(frame)
    annotated = results[0].plot()  # ← стандартная отрисовка YOLO
    cv2.imwrite("test_frame.jpg", annotated)
cap.release()


0: 384x640 12 persons, 1 stop sign, 2 umbrellas, 78.8ms
Speed: 2.3ms preprocess, 78.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


видно что есть лишние объекты и слишком жирная рамка

In [21]:
import cv2

cap = cv2.VideoCapture("../crowd.mp4")

if not cap.isOpened():
    raise IOError("Cannot open video file")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Resolution: {width}x{height}")
print(f"FPS: {fps}")
print(f"Total frames: {frame_count}")

cap.release()

Resolution: 1920x1080
FPS: 29.97002997002997
Total frames: 705


In [23]:
from ultralytics import YOLO
import cv2

model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture("../crowd.mp4")
ret, frame = cap.read()

if not ret:
    raise ValueError("Failed to read frame from video!")

results = model(frame, imgsz=1280)
boxes = results[0].boxes

annotated_frame = frame.copy()

for box in boxes:
    cls_id = int(box.cls.item())
    conf = float(box.conf.item())
    xyxy = box.xyxy[0].cpu().numpy()

    if cls_id == 0 and conf > 0.1:
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
        label = f"person {conf:.2f}"
        cv2.putText(
            annotated_frame,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.5,
            color=(0, 255, 0),
            thickness=1,
            lineType=cv2.LINE_AA
        )

cv2.imwrite("annotated_frame.jpg", annotated_frame)
cap.release()

print("✅ Frame processed and saved as 'annotated_frame.jpg'")


0: 736x1280 21 persons, 1 bus, 3 umbrellas, 3 handbags, 1 tv, 207.4ms
Speed: 9.3ms preprocess, 207.4ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)
✅ Frame processed and saved as 'annotated_frame.jpg'


# видеопоток


Урежем видео для экспериметов

In [1]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Open input video
input_video_path = "./crowd_5s.mp4"  # Убедитесь, что файл лежит в рабочей директории
capture = cv2.VideoCapture(input_video_path)

if not capture.isOpened():
    raise IOError(f"Cannot open video file: {input_video_path}")

# Get video properties
fps = int(capture.get(cv2.CAP_PROP_FPS))
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Используем AVI для надёжности
output_video_path = "../detect.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame
while True:
    ret, frame = capture.read()
    if not ret:
        break

    # Run inference with higher resolution for better accuracy on 1080p video
    results = model(frame)
    boxes = results[0].boxes

    annotated_frame = frame.copy()

    for box in boxes:
        cls_id = int(box.cls.item())
        conf = float(box.conf.item())
        xyxy = box.xyxy[0].cpu().numpy()

        # Only detect 'person' (class ID 0 in COCO)
        if cls_id == 0 and conf > 0.3:  # reasonable confidence threshold
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
            label = f"person {conf:.2f}"
            cv2.putText(
                annotated_frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.5,
                color=(0, 255, 0),
                thickness=1,
                lineType=cv2.LINE_AA,
            )

    writer.write(annotated_frame)

# Release resources
capture.release()
writer.release()

print(f"✅ Processing complete. Output saved as '{output_video_path}'")


WARNING ⚠️ torchvision==0.24 is incompatible with torch==2.7.
Run 'pip install torchvision==0.22' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation

0: 384x640 12 persons, 1 stop sign, 2 umbrellas, 201.9ms
Speed: 10.2ms preprocess, 201.9ms inference, 14.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 1 stop sign, 1 umbrella, 1 handbag, 79.8ms
Speed: 2.7ms preprocess, 79.8ms inference, 6.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 stop sign, 2 umbrellas, 1 handbag, 64.4ms
Speed: 2.2ms preprocess, 64.4ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 stop sign, 3 umbrellas, 1 handbag, 126.3ms
Speed: 5.4ms preprocess, 126.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 1 stop sign, 2 umbrellas, 98.7ms
Speed: 6.2ms preprocess, 98.7ms infe

In [ ]:
import subprocess
import os
subprocess.run(["ffmpeg", "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

print("🔧 Converting to H.264 MP4 for compatibility...")
result = subprocess.run([
    "ffmpeg", "-y", "-i", output_video_path,
    "-c:v", "libx264", "-crf", "23", "-pix_fmt", "yuv420p", "-preset", "fast",
    output_video_path
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

NameError: name 'subprocess' is not defined

In [6]:
import os
print(os.path.exists("crowd.mp4"))  # Должно быть True

False


In [8]:
%pwd

'/home/demon/ML/recognize_people/Detecting-people/notebooks'